In [1]:
# First, convert the json into python dictionary
import json
    
## read the json file
f = open('C:\\\\Users\\\\mingh\\\\Desktop\\\\text\\\\Master\\\\Thesis\\\\Python Code\\\\Algorithm 2\\\\shop_dict_A2.json')
data = json.load(f)

In [2]:
# datatype score

def datatype_Score(key1,key2):
    if key1['keytype'] != key2['keytype']:
        pairDatatype = 'unmatched'
        return pairDatatype
    else:
        pairDatatype = key1['keytype'] 
        return pairDatatype

In [3]:
# diversity score
## punish very small diversity 
def div_Score(key1,key2):
    if key1['Diversity'] == 1 or key2['Diversity'] == 1:
        divScore = -1 
        return divScore
    else:
        divScore = 0
        return divScore

In [4]:
# value similarity --- string (Qgram)


from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
from nltk.metrics.distance  import edit_distance

def qgram_similarity(string1,string2,q=3):
    similarity = 1-jaccard_distance(set(ngrams(string1, q)), set(ngrams(string2, q)))
    return(similarity)

In [5]:
def string_score(key1,key2,q):   
    string_1 = '_'.join(sorted(set(key1['String Values'])))
    string_2 = '_'.join(sorted(set(key2['String Values'])))
    
    if len(string_1) > q and len(string_2) >q :
        stringScore = qgram_similarity(string_1,string_2,q)
        return(stringScore)
    else:
        stringScore = qgram_similarity(string_1,string_2,1)
        return(stringScore)

In [6]:
# value similarity --- double

from scipy import stats

def double_similarity(double_key1,double_key2,key1_Dimension,key2_Dimension):
    if key1_Dimension == key2_Dimension:
    #dimension are same
        if key1_Dimension > 1:
            P_similarity = []
            # prepare the dataset only contains correct dimensions
            value_key1 = []
            for i in double_key1:
                if len(i) == key1_Dimension:
                    value_key1.append(i)
                else: pass
            value_key2 = []
            for i in double_key2:
                if len(i) == key1_Dimension:
                    value_key2.append(i)
                else: pass 
                
            position = 0
            while position < key1_Dimension:
                #creat the parameter
                global value_in_key1
                global value_in_key2
                value_in_key1 = []
                value_in_key2 = []
                for i in double_key1:
                    if len(i) == key1_Dimension:
                        value_in_key1.append(i[position])
                    else: pass
                for i in double_key2:
                    if len(i) == key2_Dimension:
                        value_in_key2.append(i[position])
                    else: pass
                    
                t_value,p_value = stats.ttest_ind(value_in_key1,value_in_key2)
                    
                P_similarity.append(p_value)
                position += 1
            
            double_Score = sum(P_similarity)/len(P_similarity)

            return(double_Score)
        elif key1_Dimension == 1:
            t_value,p_value = stats.ttest_ind(value_in_key1,value_in_key2)
            return(p_value)
        else: return(0)
    
    else:      
    #dimension are different
        return(0) 


In [7]:
# is string

def is_string(key):
    if key['keytype'] == 'double':
        return(0)
    else: return(1)

In [8]:
# final score calculation
def final_score(nameScore,divScore,doubleScore,stringScore,isString):
    w_name = 0.7
    w_div = 0.05
    w_double = 0.1
    w_string = 0.1
    w_is_string = 0.05
    
    final_score = nameScore*w_name + divScore*w_div + doubleScore*w_double + stringScore*w_string + isString*w_is_string
    
    return(final_score)
    # how to weight agerage all the index?

In [15]:
# record the matched keys 
from collections import defaultdict


## define the dict filter
dict_filter = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])
matched_key = defaultdict(lambda: 'None')

for shop in data.keys():
    feature_Map = data[shop]
    rest_shops = list(data.keys())
    rest_shops.remove(shop)
    matched_key_shop = defaultdict(lambda:{rest_shops[0]:"None",rest_shops[1]:"None",rest_shops[2]:"None"})
    for rest_shop in rest_shops:
        festure_Map_other = data[rest_shop]
        
        # start comparing keys
        for key1 in feature_Map.keys():
            for key2 in festure_Map_other.keys():
                datatypeScore = datatype_Score(feature_Map[key1],festure_Map_other[key2])
                if datatypeScore == 'unmatched':
                    finalScore = -1
                else:
                    divScore = div_Score(feature_Map[key1],festure_Map_other[key2])
                    nameScore = qgram_similarity(key1,key2,3)
                    if datatypeScore == 'double':
                        doubleScore = double_similarity(feature_Map[key1]['Double Values'],festure_Map_other[key2]['Double Values'],
                                                        feature_Map[key1]['Dimension'],feature_Map[key2]['Dimension'])
                        stringScore = 0
                    elif datatypeScore == 'string':
                        stringScore = string_score(feature_Map[key1],festure_Map_other[key2],3)
                        doubleScore = 0
                        
                    isStringScore = is_string(feature_Map[key1])

                    try:
                        finalScore = final_score(nameScore,divScore,doubleScore,stringScore,isStringScore)
                    except:
                        finalScore = 0
                        print(key1,key2)
                
                
                if matched_key_shop[key1][rest_shop] == 'None':
                    matched_key_shop[key1][rest_shop] = {'key':key2,'score':finalScore}
                    
                elif matched_key_shop[key1][rest_shop]['score']<finalScore:
                    matched_key_shop[key1][rest_shop]['key'] = key2
                    matched_key_shop[key1][rest_shop]['score'] = finalScore
                    matched_key_shop[key1][rest_shop]['datatype'] = feature_Map[key1]['keytype']
                    
                elif matched_key_shop[key1][rest_shop]['score']<finalScore:
                    pass
                
    matched_key[shop] = matched_key_shop



In [16]:
matched_key['bestbuy.com']

defaultdict(<function __main__.<lambda>()>,
            {'Meta Title': {'newegg.com': {'key': 'Meta Title',
               'score': 0.85,
               'datatype': 'string'},
              'amazon.com': {'key': 'Meta Title',
               'score': 0.85,
               'datatype': 'string'},
              'thenerds.net': {'key': 'Meta Title',
               'score': 0.85,
               'datatype': 'string'}},
             'Energy Star Compliant': {'newegg.com': {'key': 'Energy Star Compliant',
               'score': 0.85,
               'datatype': 'string'},
              'amazon.com': {'key': 'Energy Star Compliant',
               'score': 0.85,
               'datatype': 'string'},
              'thenerds.net': {'key': 'Energy Star Compliant',
               'score': 0.85,
               'datatype': 'string'}},
             'Width': {'newegg.com': {'key': 'Width',
               'score': 0.7999999999999999,
               'datatype': 'double'},
              'amazon.com': {'key'

In [17]:
import json
with open('matched_mape.json', 'w') as f:
    json.dump(matched_key, f)